In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

## function definitions

In [ ]:
def poly_G(v):
    # Precompute common polynomial factor
    # input v: vector of evaluation points
    # output G: polynomial in F[Y]
    
    G = PF(1)
    for i in range(len(v)):
        G *= Y-v[i]
    return G

In [ ]:
def poly_L(i,G,v):
    # Precompute Lagrangian polynomial using
    # the common polynomial from G = poly_G(v)
    
    # input i: index of the point in v where the evaluation gives 1
    #       G: product of all the factors (Y - v[i])
    #       v: vector of evaluation points
    # output L_i: polynomial in F[Y]
    
    L_i = G/(Y-v[i])
    denom = F(1)
    for h in range(len(v)):
        if h == i:
            continue
        denom *= v[i]-v[h]
    L_i /= denom
    return L_i

In [ ]:
def sys_G_RS():
    # Compute a very systematic generator matrix as in Proposition 4.7
    # output G_RS: k x n matrix with elements in F 
    
    eye = matrix.identity(F,k)
    G_RS = eye.augment(matrix.zero(F,nrows=k,ncols=n-k))
    
    G = poly_G(x[:k])
    for i in range(k):
        L_i = poly_L(i,G,x[:k]) # Lagrange
        G_RS[i,k:] = matrix(F,[L_i(x[j]) for j in range(k,n)])
    
    return G_RS

In [ ]:
def RS_decoder(r):
    # Fast interpolation decoding
    # G_poly and L are precomputed globally:
    # G_poly = poly_G(x)
    # L[i] = L_i = poly_L(i,G,x)
    # input r: received message (vector)
    # output : vector of elements in F, the codeword found by fast interpoation decoding
    
    R = r*L
    P = matrix(PF, [[G_poly,0],[-R,1]]) # initial matrix P
    
    while P[1,0].degree() >= (P[1,1].degree() + k - 1): # check if P is k_0 reduced
        (qP, rP) = P[0][0].quo_rem(P[1][0]) # polynomial division
        P = matrix(PF, [ [P[1,0], P[1,1]], [rP, P[0,1] - qP*P[1,1]] ])
    
    if P[0,0].degree() < P[1,1].degree() + (k-1): # find Q0 and Q1 in the row space of P
        f = -P[0,0]/P[0,1]
    else:
        f = -P[1,0]/P[1,1]
        
    return vector(F,[f(x[i]) for i in range(n)])


In [ ]:
def sim_chan(p,c):
    # simulate errors occuring in the symmetric channel
    # input p: cross-over probability
    #       c: codeword sent through channel
    # output r: received codeword
    # global variable alphabet = F.list()
    
    r = vector(F,n)
    for i in range(n):
        if random.random() > p:
            r[i] = c[i]
        else:
            beta_idx = (alphabet.index(c[i]) + random.randint(1,q-1)) % q
            r[i] = alphabet[beta_idx]
    return r

In [ ]:
def random_msg():
    # generate a pseudo-random message
    # ouput: vector in F^k
    
    return vector([F.random_element() for i in range(k)])

## Simulations of decoding error rate

In [ ]:
# Precomputing
q = 256; n = 255; k = 251
F.<X> = FiniteField(q)
PF.<Y> = PolynomialRing(F)
alphabet = F.list()
x = alphabet[1:]

G = sys_G_RS()

G_poly = poly_G(x)
L = vector(PF,[poly_L(i,G_poly,x) for i in range(n)])

#parity check matrix:
A = G[:,k:]
B = -A.transpose()
H = B.augment(matrix.identity(F,n-k))


In [ ]:
# probability simulation
def simulate_failure_rate(p,N):
    # simulate the encoding, and decoding after potential errors through the symmetric channel
    # input p: cross-over probability
    #       N: number of decoding to simulate
    # output p
    #        N
    #        failures: number of decoding failures
    #        errors: number of decoding errors
    
    failures = 0
    errors = 0
    for i in range(N):
        m = random_msg()
        c = m*G
        r = sim_chan(p,c)
        c_hat = RS_decoder(r)
        if c_hat != c:
            failures += 1
            if H*c_hat == vector(F,n-k):
                errors += 1
                
        if (i+1) % 100 == 0:
            print(f"{100*(i+1)/N}% : {failures}, {errors}")
            
    print(f"Number of failures: {failures}")
    print(f"Number of errors: {errors}")
    print(f"Failure rate: {failures/N}")
    print(f"Error rate: {errors/N}")
    return (p,N,failures,errors)

In [ ]:
test1 = simulate_failure_rate(0.001,1000)

In [ ]:
test2 = simulate_failure_rate(0.001,1000)

In [ ]:
test3 = simulate_failure_rate(0.001,1000)

In [ ]:
test4 = simulate_failure_rate(0.001,1000)

In [ ]:
test5 = simulate_failure_rate(0.001,1000)

### Decoding with x errors

In [ ]:
def add_x_errors(x,c):
    # simulate x errors on a codeword
    # input x: number of errors to add
    #       c: a codeword (vector)
    # output c_err: vector with x different elements from c
    
    indexes = random.sample(range(k),x)
    c_err = vector(F,n)
    for i in range(n):
        if i in indexes:
            primitive_power = (alphabet.index(c[i]) + random.randint(1,q-1)) % q
            c_err[i] = alphabet[primitive_power]
        else:
            c_err[i] = c[i]
    return c_err

In [ ]:
def simulate_decoding_x_errors(x,N):
    # simulate the encoding, and decoding of codewords that have x errors
    # input x: number of errors on each codeword before decoding
    #       N: number of decoding to simulate
    # output x
    #        N
    #        failures: number of decoding failures
    #        errors: number of decoding errors
    failures = 0
    errors = 0
    for i in range(N):
        m = random_msg()
        c = m*G
        r = add_x_errors(x,c)
        c_hat = RS_decoder(r)
        if c_hat != c:
            failures += 1
            if H*c_hat == vector(F,n-k):
                errors += 1
        if (i+1) % 100 == 0:
            print(f"{100*(i+1)/N}% : {failures}, {errors}")
    print(f"Simulated channel errors: {x}")
    print(f"Number of failures: {failures}")
    print(f"Number of errors: {errors}")
    print(f"Failure rate: {failures/N}")
    print(f"Error rate: {errors/N}")
    return (x,N,failures,errors)

In [ ]:
simulations = [simulate_decoding_x_errors(x,100) for x in range(5)]

In [ ]:
plot_data = matrix(simulations)
plot_data

In [ ]:
xdata = plot_data[:,0]
ydata1 = plot_data[:,2]
ydata2 = plot_data[:,3]
plt.scatter(xdata, ydata2, color = 'r')
plt.scatter(xdata, ydata1, color = 'b')